IMPORTING BASIC MODULES

In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tkinter import *
from flask import Flask, jsonify

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

DETECTIONS

In [11]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        


        #. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('FitVerse Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

results.pose_landmarks.landmark[0].visibility

AttributeError: 'NoneType' object has no attribute 'landmark'

CAPTURE COORD & CSV

In [4]:
import csv
import os
import numpy as np

In [7]:
num_coords = len(results.pose_landmarks.landmark)
num_coords

33

In [8]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [9]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [13]:
class_name = "hands down"

THE BELOW PART IS FOR RECORDING NEW CLASSES OR NEW DATASETS

In [14]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
       
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        


        #. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

         # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            row = pose_row
            row.insert(0, class_name)

            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
        except:
            pass
                        
        cv2.imshow('FitVerse Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

TRAINING MODEL

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:
df = pd.read_csv('coords.csv')


In [17]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)
y_test

644          Crunches
2509           Squats
2072       Right Lean
1504         Crunches
883       palms wrong
            ...      
124        Legs close
1021    knee straight
88              Curls
622          Crunches
1147    knee straight
Name: class, Length: 788, dtype: object

In [19]:
#Training Dependencies

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [20]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [21]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model
fit_models

c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [22]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [23]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.998730964467005
rc 0.998730964467005
rf 0.998730964467005
gb 0.998730964467005


In [69]:
fit_models['rc'].predict(X_test)
y_test

1309         Crunches
2372         Crunches
2221            Curls
1329         Crunches
1435         Crunches
            ...      
1848        Left Lean
1083    knee straight
1078    knee straight
2047       Right Lean
256        Right Lean
Name: class, Length: 722, dtype: object

In [24]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

MAKING DETECTIONS

In [25]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [26]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [27]:
def CalcAngle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)

    rad = np.arctan2(c[1]-b[1], c[0] - b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(rad*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle

In [74]:
import pyttsx3    #For text to speech conversion

cap = cv2.VideoCapture(0)

count_curl = 0
stage_curl = None

count_crunch = 0
stage_crunch = None 

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        


        # . Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            

            # Concate rows
            row = pose_row
            
        
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            

           ################################## C U R L  ################################
          
            if body_language_class == 'Curls':
                #Getting Coordinates

                shoulder = [pose[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x, pose[mp_holistic.PoseLandmark.LEFT_SHOULDER].y]
                elbow = [pose[mp_holistic.PoseLandmark.LEFT_ELBOW.value].x, pose[mp_holistic.PoseLandmark.LEFT_ELBOW].y]
                wrist = [pose[mp_holistic.PoseLandmark.LEFT_WRIST.value].x, pose[mp_holistic.PoseLandmark.LEFT_WRIST].y]

                angle= CalcAngle(shoulder, elbow, wrist)

                cv2.putText(image, str(angle), tuple(np.multiply(elbow, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),2, cv2.LINE_AA)
                
                #Curl Counter Logic

                if angle > 160:
                    stage_curl = "down"
                if angle < 7 and stage_curl == 'down':
                    stage_curl = "up"
                    count_curl +=1
                    print(count_curl)

                cv2.rectangle(image, (0,0), (255,73), (200,67,87), -1)
                
                cv2.putText(image, "Curls:", (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),1 , cv2.LINE_AA)

                cv2.putText(image, str(count_curl), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255),2, cv2.LINE_AA) 

            if body_language_class == 'Legs close':
                text_speech = pyttsx3.init()

                legs_close = "Please make shoulder distance between your legs"
                text_speech.say(legs_close)
                text_speech.runAndWait()
            
            if body_language_class == 'Right Lean':
                text_speech = pyttsx3.init()

                right_lean = "You are leaned too much right keep your body straight"
                text_speech.say(right_lean)
                text_speech.runAndWait()

            if body_language_class == 'Left Lean':
                text_speech = pyttsx3.init()

                left_lean = "You are leaned too much left keep your body straight"
                text_speech.say(left_lean)
                text_speech.runAndWait()
            
            if body_language_class == 'Bent':
                text_speech = pyttsx3.init()

                bent = "Your back is bent stand straight"
                text_speech.say(bent)
                text_speech.runAndWait()

            if body_language_class == 'Arm sideways':
                text_speech = pyttsx3.init()

                arm = "Your arms are sideways keep them aligned to your body"
                text_speech.say(arm)
                text_speech.runAndWait()




        #######################################################################################################

 

        ####################################### C R U N C H E S ###############################################

            if body_language_class == 'Crunches':
                #Getting Coordinates

                shoulder = [pose[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x, pose[mp_holistic.PoseLandmark.LEFT_SHOULDER].y]
                hip = [pose[mp_holistic.PoseLandmark.LEFT_HIP.value].x, pose[mp_holistic.PoseLandmark.LEFT_HIP].y]
                knee = [pose[mp_holistic.PoseLandmark.LEFT_KNEE.value].x, pose[mp_holistic.PoseLandmark.LEFT_KNEE].y]

                angle= CalcAngle(shoulder, hip, knee)

                cv2.putText(image, str(angle), tuple(np.multiply(hip, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),2, cv2.LINE_AA)
           

            #Crunches Counter Logic

                if angle > 90:
                    stage_crunch = "down"
                if angle < 70 and stage_crunch == 'down':
                    stage_crunch = "up"
                    count_crunch +=1
                    print(count_crunch)

                cv2.rectangle(image, (0,0), (255,73), (200,67,87), -1)
                
                cv2.putText(image, "Crunches:", (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),1 , cv2.LINE_AA)

                cv2.putText(image, str(count_crunch), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255),2, cv2.LINE_AA)
            

            if body_language_class == 'feets close':
                text_speech = pyttsx3.init()

                feet_apart = "keep your feets apart"
                text_speech.say(feet_apart)
                text_speech.runAndWait()

            if body_language_class == 'palms wrong':
                text_speech = pyttsx3.init()

                palm = "keep your palms behind your head"
                text_speech.say(palm)
                text_speech.runAndWait()

            if body_language_class == 'knee straight':
                text_speech = pyttsx3.init()

                knee_straight = "please bend your knees"
                text_speech.say(knee_straight)
                text_speech.runAndWait()


        except:
            pass


        
                        
        cv2.imshow('FitVerse Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()




c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Right Lean [0.04 0.03 0.21 0.2  0.01 0.04 0.23 0.03 0.08 0.13]


c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Right Lean [0.02 0.02 0.22 0.14 0.01 0.03 0.25 0.03 0.14 0.14]


c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

Right Lean [0.01 0.01 0.11 0.04 0.03 0.07 0.43 0.   0.12 0.18]


c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Right Lean [0.01 0.01 0.09 0.03 0.04 0.07 0.43 0.01 0.17 0.14]


c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Right Lean [0.01 0.01 0.07 0.04 0.03 0.07 0.45 0.   0.14 0.18]


In [75]:
from werkzeug.wrappers import Request, Response
from flask import Flask

FitVerseDetections = Flask(__name__)

@FitVerseDetections.route("/", methods=['GET'])
def hello():
    return "Hello World!"

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, FitVerseDetections)

 * Running on http://localhost:5000
Press CTRL+C to quit


In [ ]:
from werkzeug.wrappers import Request, Response
from flask import Flask, jsonify
sem = "lololo"
FitverseDetections = Flask(__name__)

@FitverseDetections.route('/',methods = ['GET'])

def index():
    return jsonify({'greeetings':'Aurrrrrrrrrrrrrrr'})
    
    
if __name__ == "__main__":
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, FitVerseDetections)

In [ ]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))